## Group 6:
### AU1741001: Akash Tike
### AU1741011: Smit Mandavia
### AU1741068: Parth Maniyar
### AU1741095: Shaunak Vyas

## Collaborative filtering (Distributed stochastic gradient descent for matrix factorization)

In [0]:
from random import randint 
import random
import multiprocessing
import time
import numpy as np

Importing libaries

In [0]:
class U:
  def __init__(self, i , j ):
    self.matrix = np.random.uniform( low=0,high=1.2,size=(i, j))

  def __str__(self):  
    return 'marix U is' + self.matrix.tostring()

class V:
  def __init__(self, i , j ):
    self.matrix = np.random.uniform( low=0,high=1.2,size=(i, j))
  def __str__(self):  
    return 'marix V is' +self.matrix.tostring()

*   The matrix U and matrix V are intialized with random values in range $(0,1.2)$.

*   The range is $0$ to $1.2$ because $k=7$ and the dot product will be between 0 to $1.2*1.2*7$ ~= $10$.     
If we seed the matrix $U$ and $V$ properly they it will converge faster

*   the `__str__` function is implemented for printing the class object

In [0]:
class Rating: 
  def __init__(self, i , j , rating , block_index_i, block_index_j):
    self.i = i
    self.j = j
    self.rating = rating
    self.block_index_i = block_index_i
    self.block_index_j = block_index_j
  
  def __str__(self):  
    return "From str method of Rating:i  is % d, j is % d, blk_i is % d, blk_j is % d , rating is % d" % (self.i, self.j,self.block_index_i,self.block_index_j,self.rating)  


*   Class rating will be used to store rating updates.   
*   `i,j` denotes the user index and item index respectively.       
*   `block_index_i` and `block_index_i` denotes the index of block in which they lie after partitioning the matrix into $d*d$ smaller matrices.

In [0]:
class Turn:
  def __init__(self, p , q ):
    self.p = p
    self.q = q

  def __str__(self):  
    return "From str method of Turn: p is % d, q is % d" % (self.p, self.q)  


*   Class Turn will be used give and close turn of a worker and it will also contain information about which block is to be updated.   
*   `p` and `q` denotes the index of block in which needs to be updated.

In [0]:
class Data: 
  def __init__(self, isP, index , data ):
    self.isP = isP
    self.index = index
    self.data = data

  def __str__(self):  
    return "From str method of Data: isP is % d, index is % d , data is " % (self.isP, self.index)  


*   Class Data will be used transfer data between worker and matrix store.        
*   `isP` is `True` if block is a U type block and it is `False` if it is a V type block.   

In [0]:
class Request: 
  def __init__(self, isP, index, my_index):
    self.isP = isP
    self.index = index
    self.my_index = my_index
  def __str__(self):  
    return "From str method of Request: isP is % d, index is % d , my_index is %d " % (self.isP, self.index, self.my_index)   


*   Class Request will be used to request data from matrix store.        
*   `isP` is `True` if requested block is a U type block and it is `False` if it is a V type block.   
*   `index` denotes the index of the requested block.
*   `my_index` denotes the index of the worker who is requesting the data


In [0]:
def matrix_store(rating_matrix, is_blocked_U, is_blocked_V , matrix_U, matrix_V , matrix_store_request , matrix_store_data_send , matrix_store_data_update, command) :

  while True:
    while matrix_store_data_update.qsize()>0:     
      data = matrix_store_data_update.get()
    
      if data.isP:
        matrix_U[data.index].matrix = data.data
        is_blocked_U[data.index]=False
      else:
        matrix_V[data.index].matrix = data.data
        is_blocked_V[data.index]=False
      
    while command.qsize()>0:
      cmd = command.get()
      if cmd == 'Exit': 
        return
      if cmd == 'Print':
        prediction = np.dot(matrix_U[0].matrix.T,matrix_V[0].matrix)
        print(prediction)  
        print(rating_matrix[0][0])
        sum_of_error=0
        sum_of_square=0
        cnt = 0
        for i in range(len(prediction)):
            for j in  range(len(prediction[0])):
                if rating_matrix[0][0][i][j]!=0:
                    sum_of_error = sum_of_error+ abs( rating_matrix[0][0][i][j]-prediction[i][j]) 
                    sum_of_square = sum_of_square + abs( rating_matrix[0][0][i][j]-prediction[i][j])*abs( rating_matrix[0][0][i][j]-prediction[i][j]) 
                    cnt = cnt+1
        is_rated = (rating_matrix[0][0] != 0)*1
        print("MAE: ",np.sum(abs(rating_matrix[0][0]-prediction)*is_rated)/np.sum(is_rated))
        print("MSE: ",np.sum((np.square(rating_matrix[0][0] - prediction)*is_rated))/np.sum(is_rated),flush=True)
        print("MAE: ", sum_of_error/cnt )
        print("MSE: ", sum_of_square/cnt )
          

    if matrix_store_request.qsize()>0:
      req =  matrix_store_request.get()
      if req.isP:
          if is_blocked_U[req.index]:
             matrix_store_request.put(req)  
             continue
      else:
          if is_blocked_V[req.index]:
             matrix_store_request.put(req)  
             continue

      if req.isP:
        matrix_store_data_send[req.my_index].put(Data(True, req.index, matrix_U[req.index].matrix ))
        is_blocked_U[req.index]=True
      else:
        matrix_store_data_send[req.my_index].put(Data(False, req.index, matrix_V[req.index].matrix))
        is_blocked_V[req.index]=True
   


*   The first while loop updates the martrices received from worker after updating. It also unblocks the blocks for further updates. 
*   The second while loop is made for debugging purpose. We can give various commands to the store from main fucntion and control the flow of code. Currently `Print` command prints predicted ratings, actual ratings as well as mean absolute and mean square error. 
*   Third is if condition instead of loop. It is because the least important task is assigning matrix to new workers as comapared to other two tasks. Store cannot get overwhelmed by number of requests of data and can respond to other two tasks. 
*   Third part of code allots martrix as requested by workers and also protects similtaneous updates by blocking matrix allocated. It cannot cause a deadlock because main function also takes care of protecting data simultaneous update. *   This extra protection is just for the cases when data store is slow in updating received data from workers and main function thinks the worker already stopped working.


In [0]:
def worker( is_updated, rating_matrix , turn, turn_over, matrix_store_request , matrix_store_data_recieve , matrix_store_data_update , command):
  while True: 
    while command.qsize()>0:  # its use is same as command used in matrix store
      cmd = command.get()
      if cmd == 'Exit': 
        return
      
    my_turn  = turn.get()  # waiting for turn

    # Requesting data from data store
    matrix_store_request.put(Request(True,my_turn.p,my_turn.p))
    matrix_store_request.put(Request(False,my_turn.q,my_turn.p))   
    
    # Waiting for data (U and V) from data store
    data = matrix_store_data_recieve.get()
    data2 = matrix_store_data_recieve.get()
    
    if data.isP:  # assigns correct matrix to correct variable
      U_data = data.data
      V_data = data2.data 
    else :    
      U_data = data2.data
      V_data = data.data 


    # Caculating prediction before updating U and V

    is_rated = (rating_matrix[my_turn.q] != 0)*1
    new_prediction = np.dot(U_data.T,V_data) 
    
    if np.sum(is_rated)>0 :  # Print MAE and MSE before updates
      print(my_turn.p, ' ', my_turn.q, " Before MAE: ",np.sum(abs(rating_matrix[my_turn.q]-new_prediction)*is_rated)/np.sum(is_rated))
      print(my_turn.p, ' ', my_turn.q, " Before MSE: ",np.sum((np.square(rating_matrix[my_turn.q] - new_prediction)*is_rated))/np.sum(is_rated),flush=True)
    else : 
      print(my_turn.p, ' '  ,my_turn.q, "Empty",flush=True)  # There are no ratings in given block. This can happen if worker is slow in updting rating received.
      turn_over.put(my_turn)
      matrix_store_data_update.put(data)
      matrix_store_data_update.put(data2)
      continue  # we will skip updating as there is nothing to update
    # the main function will definitely call this once more( even if there are no futher updates) if this is empty 
    # because the sends acknowlegment for ratings received 


    # Now updating matrix using normal algoithm on assigned block of martix as implemented and discussed in 5th assignment

    #learning rate
    eta = 0.0075

    #regularization parameter to control overfitting
    lamda = 10
  
    epoch=1
    
    for itr in range(epoch):
      for i in range(len(U_data[0])):
        for j in range(len(V_data[0])):
          if rating_matrix[my_turn.q][i][j]!=0:    
            new_Ui = U_data[:,i] - 2*eta*((np.dot(U_data[:,i],V_data[:,j])-rating_matrix[my_turn.q][i][j])*V_data[:,j] + (lamda/len(V_data[0]))*U_data[:,i])
            new_Vj = V_data[:,j] - 2*eta*((np.dot(U_data[:,i],V_data[:,j])-rating_matrix[my_turn.q][i][j])*U_data[:,i] + (lamda/len(U_data[0]))*V_data[:,j])
                              
            U_data[:,i] = new_Ui
            V_data[:,j] = new_Vj

    # Completed updates successfully 

    # Calculating new predictions 
    new_prediction = np.dot(U_data.T,V_data)
    
    # Printing MAE and MSE after updates, also prints block index so that we can compare (with before update MAE and MSE) and identify accuracy of each block
    if np.sum(is_rated)>0 :
      print(my_turn.p, ' ', my_turn.q," After MAE: ",np.sum(abs(rating_matrix[my_turn.q]-new_prediction)*is_rated)/np.sum(is_rated))
      print(my_turn.p,' ', my_turn.q, " After MSE: ",np.sum((np.square(rating_matrix[my_turn.q] - new_prediction)*is_rated))/np.sum(is_rated),flush=True)
    else : # This should't happen at all ( /0 error)
      print(my_turn.p,' ', my_turn.q, "Error",flush=True)
        
    if data.isP: 
      data.data = U_data 
      data2.data = V_data   
    else :    
      data2.data = U_data 
      data.data = V_data  
    
    # Sending data back to marix store
    matrix_store_data_update.put(data)
    matrix_store_data_update.put(data2)
    
    # Complete turn by notifying master 
    turn_over.put(my_turn)

In [0]:
def master( worker_turn, worker_turn_over, d, command ) : 
  
  for itr in range(d):  # We will update (itr, pi(i) ) in each iteration to cover full matrix

    while command.qsize()>0:  # its use is same as command used in matrix store
      cmd = command.get()
      if cmd == 'Exit': 
        return

    for i in range(d):
       turn = Turn(i,(i+itr)%d)  
       worker_turn[i].put(turn) # Give work to ith worker -> update (itr, pi(i) )
      
    for i in range(d):
        turn=worker_turn_over.get() # Wait for ith worker to finish update
    time.sleep(1)  # Wait for small amount of time to ensure matrix store also finishes its work of updating matrix  
  return  

In [0]:
if __name__ == '__main__':

  d=10 # number of workers
  n=500 # number of users
  m=100 # number of items
  n_by_d = n//d  # number of users in each block
  m_by_d = m//d  # number of items in each block
  k=7    # number of latent features


In [0]:
  worker_turn_over =  multiprocessing.Queue()
  matrix_store_request = multiprocessing.Queue()
  matrix_store_data_update = multiprocessing.Queue()
  
  matrix_store_data_send = [ multiprocessing.Queue() for i in range(d) ]
  worker_turn =  [ multiprocessing.Queue() for i in range(d) ]
  worker_command = [ multiprocessing.Queue() for i in range(d) ]

  master_command = multiprocessing.Queue()
  matrix_store_command = multiprocessing.Queue()

*   Creating all queues for communication bewteen all processes 

In [0]:
  rating_matrix = [[np.zeros((n_by_d,m_by_d)) for x in range(d)] for y in range(d)]
  is_updated = [[np.zeros((n_by_d,m_by_d)) for x in range(d)] for y in range(d)]
  
  is_blocked_row =  [False for i in range(d)] 
  is_blocked_column =  [False for i in range(d)] 
  
  matrix_U = [ U(k,n_by_d)  for i in range(d) ]
  matrix_V = [ V(k,m_by_d)  for i in range(d) ]


*  Creating and intializing different matrix required

In [0]:
  for i in range(n):
    items = random.randint(0,m//2)
    for j in range(items):
      j_cor = random.randint(0,m-1);
      rating_matrix[(i//n_by_d)][(j_cor//m_by_d)][i%n_by_d][j_cor%m_by_d]=random.randint(1,10)

*   Assigning ratings to few items by each users ( approx 20% of data is assigned)

In [0]:
  master_process = multiprocessing.Process(target=master, args=( worker_turn, worker_turn_over, d, master_command))
  
  worker_process =[ multiprocessing.Process(target=worker, args=(is_updated[i], rating_matrix[i] , worker_turn[i], worker_turn_over, matrix_store_request , matrix_store_data_send[i] , matrix_store_data_update , worker_command[i])) for i in range(d) ]
  
  matrix_store_process = multiprocessing.Process(target=matrix_store, args=(rating_matrix , is_blocked_row,is_blocked_column,matrix_U, matrix_V , matrix_store_request , matrix_store_data_send , matrix_store_data_update, matrix_store_command))     


*  Creating all the processes and also passing required data, queues and matrices to them. 

In [0]:
  # Start all processes. We should start them according to topological ordering. 
  
  matrix_store_process.start()
  for i in range(d):
    worker_process[i].start() 
  master_process.start()
 
  # Wait for master to finish the work
  
  master_process.join()

  # Print prediction and actual rating for a random block 
  # and also print mean square and mean absolute error 
  # to ensure if data is not getting overfitted for a particular block during update

  matrix_store_command.put('Print')
  print('Main done',flush=True)

  # wait for 1 sec to ensure output gets printed by matrix store
  time.sleep(1)

  # End all the processes in reverse topological order

  master_command.put('Exit')
  for i in range(d):
    worker_command[i].put('Exit') 
  matrix_command.put('Exit')

0   0  Before MAE:  3.590899750478612
3   3  Before MAE:  3.3929243379506127
1   1  Before MAE:  3.3540609130955445
3   3  Before MSE:  17.46715847076283
1   1  Before MSE:  17.10217533311423
0   0  Before MSE:  19.0595906186495
3   3  After MAE:  2.5242992156121544
1   1  After MAE:  2.6875180927569775
0   0  After MAE:  2.6636802014186856
1   1  After MSE:  10.981001991264076
2   2  Before MAE:  3.119454508183493
0   0  After MSE:  10.335599763018886
4   4  Before MAE:  3.6773207972283757
2   2  Before MSE:  15.719848807518565
3   3  After MSE:  9.708807296872239
4   4  Before MSE:  19.552940390493205
5   5  Before MAE:  3.3560790633559705
6   6  Before MAE:  3.354985304399271
4   4  After MAE:  2.7523142790205815
2   2  After MAE:  2.437485862378393
7   7  Before MAE:  3.6323365805137446
4   4  After MSE:  10.87754234702587
5   5  Before MSE:  16.975770539726117
9   9  Before MAE:  3.381776178019585
2   2  After MSE:  9.549214026501385
8   8  Before MAE:  3.1197445983654295
6   6  B